# Problem 3: MNIST Sample

In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.MNIST)

In [3]:
path.ls()

(#2) [Path('/Users/lberelidze/.fastai/data/mnist_png/training'),Path('/Users/lberelidze/.fastai/data/mnist_png/testing')]

In [4]:
fours = (path / 'training' / '4').ls().sorted()
nines = (path / 'training' / '9').ls().sorted()

In [5]:
Image.open(fours[1])

In [6]:
Image.open(nines[2])

In [7]:
train_4_tens = torch.stack([tensor(Image.open(o)) for o in (path/'training'/'4').ls()])
train_4_tens = train_4_tens.float()/255
train_9_tens = torch.stack([tensor(Image.open(o)) for o in (path/'training'/'9').ls()])
train_9_tens = train_9_tens.float()/255

In [8]:
valid_4_tens = torch.stack([tensor(Image.open(o)) for o in (path/'testing'/'4').ls()])
valid_4_tens = valid_4_tens.float()/255
valid_9_tens = torch.stack([tensor(Image.open(o)) for o in (path/'testing'/'9').ls()])
valid_9_tens = valid_9_tens.float()/255

In [9]:
train_x = torch.cat([train_4_tens, train_9_tens]).view(-1, 28 * 28)
train_y = tensor([1] * len(train_4_tens) + [0] * len(train_9_tens)).unsqueeze(1)
dset = list(zip(train_x,train_y))

In [10]:
valid_x = torch.cat([valid_4_tens, valid_9_tens]).view(-1, 28 * 28)
valid_y = tensor([1] * len(valid_4_tens) + [0] * len(valid_9_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [11]:
dl = DataLoader(dset, batch_size=256)

In [12]:
valid_dl = DataLoader(valid_dset, batch_size=256)

In [13]:
dls = DataLoaders(dl, valid_dl)

In [14]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [15]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()

In [123]:
def train(activation):
    simple_net = nn.Sequential(
        nn.Linear(28 * 28, 30),
        activation,
        nn.Linear(30, 1)
    )
    learn = Learner(dls, simple_net, opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)
    print(activation.__class__.__name__)
    learn.fit(40, 0.1)

In [60]:
stats = []

In [124]:
train(nn.ReLU())

ReLU


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.550572,0.473926,0.622300,00:00
1,0.432723,0.462696,0.506781,00:00
2,0.296933,0.454565,0.510799,00:00
3,0.203898,0.419272,0.542441,00:00
4,0.142038,0.377111,0.593169,00:00
5,0.103455,0.337833,0.641386,00:00
6,0.079926,0.302573,0.688599,00:00
7,0.065541,0.272088,0.727775,00:00
8,0.056633,0.246414,0.757408,00:00
9,0.050945,0.225092,0.784028,00:00


In [61]:
stats.append(['ReLU', 0.030485, 0.080378, 0.930186])

In [128]:
train(nn.Sigmoid())

Sigmoid


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.587402,0.496202,0.493219,00:00
1,0.578368,0.490370,0.499749,00:00
2,0.562832,0.484008,0.507785,00:00
3,0.537702,0.478000,0.506781,00:00
4,0.501329,0.472659,0.506781,00:00
5,0.456329,0.467009,0.506781,00:00
6,0.408850,0.458743,0.506781,00:00
7,0.362267,0.444869,0.506781,00:00
8,0.315527,0.423389,0.512808,00:00
9,0.269595,0.394982,0.553491,00:00


In [62]:
stats.append(['Sigmoid', 0.055076, 0.077676, 0.939729])

In [129]:
train(nn.Tanh())

Tanh


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.577293,0.456408,0.589151,00:00
1,0.438558,0.452027,0.506781,00:00
2,0.294089,0.436380,0.520342,00:00
3,0.198098,0.391111,0.578101,00:00
4,0.136445,0.344411,0.633350,00:00
5,0.098898,0.303056,0.688599,00:00
6,0.076561,0.268180,0.735811,00:00
7,0.063292,0.239819,0.768960,00:00
8,0.055229,0.217109,0.791562,00:00
9,0.050102,0.198747,0.817177,00:00


In [63]:
stats.append(['Tanh', 0.030697, 0.076838, 0.931693])

In [130]:
train(nn.LeakyReLU())

LeakyReLU


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.532174,0.461828,0.506781,00:00
1,0.385141,0.466754,0.506781,00:00
2,0.269485,0.447115,0.512305,00:00
3,0.185582,0.407616,0.559016,00:00
4,0.130400,0.365261,0.609242,00:00
5,0.096137,0.326543,0.657459,00:00
6,0.075192,0.292672,0.703164,00:00
7,0.062393,0.263783,0.738322,00:00
8,0.054468,0.239390,0.765947,00:00
9,0.049393,0.219115,0.788548,00:00


In [64]:
stats.append(['LeakyReLU', 0.030231, 0.079682, 0.930186])

In [131]:
train(nn.ELU())

ELU


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.551411,0.441948,0.515319,00:00
1,0.385339,0.451618,0.508790,00:00
2,0.258210,0.425995,0.530387,00:00
3,0.174534,0.378920,0.589653,00:00
4,0.121859,0.334656,0.647413,00:00
5,0.090177,0.296638,0.692617,00:00
6,0.071398,0.264813,0.734807,00:00
7,0.060191,0.238528,0.770467,00:00
8,0.053293,0.217006,0.795078,00:00
9,0.048822,0.199297,0.813159,00:00


In [66]:
stats.append(['ELU', 0.030637, 0.077772, 0.931693])

In [132]:
class Swish(nn.Module):
        def __init__(self, b = 1.):
            super(Swish, self).__init__()
            self.b = b

        def forward(self, x):
            sigmoid = torch.sigmoid(x) ** self.b
            return x * sigmoid

In [133]:
train(Swish())

Swish


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.548247,0.478522,0.822200,00:00
1,0.475972,0.438790,0.517328,00:00
2,0.339888,0.442828,0.513812,00:00
3,0.230889,0.411510,0.554998,00:00
4,0.156833,0.366577,0.609242,00:00
5,0.110566,0.326090,0.657961,00:00
6,0.082988,0.291198,0.698142,00:00
7,0.066679,0.261822,0.737820,00:00
8,0.056882,0.237659,0.764942,00:00
9,0.050784,0.217882,0.789553,00:00


In [67]:
stats.append(['Swish', 0.029687, 0.082521, 0.927172])

In [69]:
import pandas as pd

In [71]:
pd.DataFrame(stats, columns=['Activation', 'Training Loss', 'Validation Loss', 'Accuracy'])

,Activation,Training Loss,Validation Loss,Accuracy
0,ReLU,0.030485,0.080378,0.930186
1,Sigmoid,0.055076,0.077676,0.939729
2,Tanh,0.030697,0.076838,0.931693
3,LeakyReLU,0.030231,0.079682,0.930186
4,ELU,0.030637,0.077772,0.931693
5,Swish,0.029687,0.082521,0.927172
